<a href="https://colab.research.google.com/github/cmoran570/prueba/blob/master/Quismodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cargando Librerias


In [1]:
!pip install pypdf==3.16.4 cohere tiktoken langchain==0.0.324 sentence-transformers==2.2.2 openai==0.28.1 pinecone-client==2.2.4 -q

In [ ]:
!pip show pinecone-client

Cargar y Guardar Documentos PDF

In [2]:
import requests

# Fichero PDF
archivos_url = ["https://preguntapdf.s3.eu-south-2.amazonaws.com/la-rosaleda_v3.pdf",
                "https://preguntapdf.s3.eu-south-2.amazonaws.com/GuiaViajeMexicoDF.pdf"]

for url in archivos_url:
    doc_to_download = requests.get(url)
    # Guardar fichero
    pdf_file = open(url.split("/")[-1], "wb")
    pdf_file.write(doc_to_download.content)

1 Pinecone

In [3]:
import pinecone
from langchain.vectorstores import Pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


#### A - Primeros pasos

In [4]:
# Inicializar

PINECONE_API_KEY = "901b68ff-f539-403f-b895-ddfb960a115c"
PINECONE_ENV = "us-east1-gcp"
INDEX_NAME = 'taller'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In [5]:
pinecone.list_indexes()

[]

In [ ]:
# Describe Index
index_description = pinecone.describe_index(INDEX_NAME)

In [ ]:
index_description.name
index_description.dimension
index_description.metric

#### B - Cargar Vectores

In [ ]:
import numpy as np

In [ ]:
vector_1 = np.random.uniform(-1, 1,384).tolist()
vector_2 = np.random.uniform(-1, 1,384).tolist()

In [ ]:
index = pinecone.Index(INDEX_NAME)

In [ ]:
upsert_response = index.upsert(
   vectors=[
       {'id': "vec1", "values":vector_1, "metadata": {'genre': 'cine'}},
       {'id': "vec2", "values":vector_2, "metadata": {'genre': 'teatro'}},
   ]
)

#### C - Buscar vectores

In [ ]:
vector_pregunta = np.random.uniform(-1, 1,384).tolist()

In [ ]:
result = index.query(queries=[vector_pregunta], top_k=1)
print(result)

{'results': [{'matches': [{'id': 'vec1', 'score': 0.00472536357, 'values': []}],
              'namespace': ''}]}


#### D - Borrar

In [ ]:
index.delete(
    filter={
        "genre": "cine"
    }
)

{}

In [ ]:
# borrar todos los vectores
index.delete(delete_all=True)

{}

### 2 -Leer PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
# Cargar fichero PDF
# FILE = 'GuiaViajeMexicoDF.pdf'
FILE = "la-rosaleda_v3.pdf"

In [ ]:
# trabajaremos con documents y no text
loader = PyPDFLoader(FILE)
doc = loader.load()

### 3 - Crear chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
    )

In [ ]:
# split_documents y no text
chunks = text_splitter.split_documents(doc)

In [ ]:
len(chunks)

7

In [ ]:
def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )
    return text_splitter.split_documents(doc_to_chunk)

### 4 - Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11G

'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("¿Cuántas copas ha ganado el Málaga CF?")

In [ ]:
# sentence_embeddings

In [ ]:
len(sentence_embeddings)

384

In [ ]:
Pinecone.from_documents(
    chunks,
    embeddings,
    index_name=INDEX_NAME)

### 5 - Preguntar al documento

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
vstore = Pinecone.from_existing_index(INDEX_NAME, embeddings)

llm = ChatOpenAI(model_name='gpt-3.5-turbo')
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
pregunta = "¿Cuántas copas ha ganado el Málaga?"
# pregunta = "¿Dónde está ubicado el monumento a la revolución?"
pregunta = "¿Cuántos habitantes hay en la ciudad de México?"

In [ ]:
# Busqueda de párrafos similares
docs = vstore.similarity_search(pregunta, 3)
# Utilizar los parrafos similares para darle contexto a ChatGPT
respuesta = chain.run(input_documents=docs, question=pregunta)
print(f"Respuesta ChatGPT: {respuesta}")

Respuesta ChatGPT: En México viven 20 millones de personas.


Añadir vectores

In [ ]:
res = vstore.from_texts(
    ["En la Ciudad de México viven 20 millones de personas"],
    embeddings,
    metadatas=[{'doc':'deporte'}],
    index_name=INDEX_NAME)

Revisar Costes

In [ ]:
from langchain.callbacks import get_openai_callback

In [ ]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=pregunta)
    print(cb)

Tokens Used: 531
	Prompt Tokens: 521
	Completion Tokens: 10
Successful Requests: 1
Total Cost (USD): $0.0008015000000000001
